<br>
<br>

# **준비 사항**

In [1]:
from google.colab import drive
drive.mount('content/')

Mounted at content/


In [3]:
# Hugging Face의 트랜스포머 모델을 설치
!pip install transformers --quiet

     |████████████████████████████████| 1.4MB 8.7MB/s 
     |████████████████████████████████| 2.9MB 24.3MB/s 
     |████████████████████████████████| 890kB 35.6MB/s 


In [4]:
import tensorflow as tf
import torch

from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime
import os
import re

<br>
<br>

# **데이터 로드**

In [5]:
# 네이버 영화리뷰 감정분석 데이터 다운로드
!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 26.94 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.


In [6]:
# 디렉토리의 파일 목록
os.listdir('nsmc/')

['ratings_test.txt',
 '.git',
 'raw',
 'README.md',
 'synopses.json',
 'code',
 'ratings_train.txt',
 'ratings.txt']

In [ ]:
filepath = 'nsmc'
train = pd.read_table(os.path.join(filepath, 'ratings_train.txt'))
test = pd.read_table(os.path.join(filepath, 'ratings_test.txt'))

print(train.shape)
print(test.shape)

(150000, 3)
(50000, 3)


In [ ]:
# 필요 패키지 다운로드
!pip install soynlp --quiet  # 반복 단어 축약 soynlp repeat_normalize() 모듈 활용 예정

     |████████████████████████████████| 419kB 14.0MB/s 


In [ ]:
# 훈련 데이터셋 전처리
train = train.dropna(how='any')  # nan 열 제거
train['document'] = train['document'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣!.~;^ ]', '')  # 한국어+공백 외 제거
train['document'].replace('', np.nan, inplace=True)  # regex에 의해 공백으로 치환된 값 nan 값으로 변경
train = train.dropna(how='any')  # nan 열 제거

# 테스트 데이터셋 전처리
test = test.dropna(how='any')  # nan 열 제거
test['document'] = test['document'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣!.~;^ ]', '')  # 한국어+공백 외 제거
test['document'].replace('', np.nan, inplace=True)  # regex에 의해 공백으로 치환된 값 nan 값으로 변경
test = test.dropna(how='any')  # nan 열 제거

# 반복 단어 축약
from soynlp.normalizer import repeat_normalize

train['document'] = train['document'].apply(lambda sentence: repeat_normalize(sentence, num_repeats=2))
test['document'] = test['document'].apply(lambda sentence: repeat_normalize(sentence, num_repeats=2))

# 중복 데이터 제거
train.drop_duplicates(subset=['document'], inplace=True)
test.drop_duplicates(subset=['document'], inplace=True)

# 파일 저장
train.to_csv('nsmc_preprocessed_train', encoding='utf-8')
test.to_csv('nsmc_preprocessed_test', encoding='utf-8')

In [ ]:
print(train.columns)
print(test.columns)
print(train.head())
print(test.head())
print(train.isnull().sum())
print(test.isnull().sum())

Index(['id', 'document', 'label'], dtype='object')
Index(['id', 'document', 'label'], dtype='object')
         id                                           document  label
0   9976970                                아 더빙.. 진짜 짜증나네요 목소리      0
1   3819312                  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
2  10265843                                  너무재밓었다그래서보는것을추천한다      0
3   9045019                      교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정      0
4   6483659  사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...      1
        id                                     document  label
0  6270596                                          굳 ㅋ      1
2  8544678         뭐야 이 평점들은.... 나쁘진 않지만 점 짜리는 더더욱 아니잖아      0
3  6825595             지루하지는 않은데 완전 막장임... 돈주고 보기에는....      0
4  6723715  만 아니었어도 별 다섯 개 줬을텐데.. 왜 로 나와서 제 심기를 불편하게 하죠      0
5  7898805                            음악이 주가 된 최고의 음악영화      1
id          0
document    0
label       0
dtype: int64
id          0
document    0
label       0
dty

In [ ]:
# BERT의 입력 형식에 맞게 변환
sentences = train['document'].values
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 아 더빙.. 진짜 짜증나네요 목소리 [SEP]',
 '[CLS] 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나 [SEP]',
 '[CLS] 너무재밓었다그래서보는것을추천한다 [SEP]',
 '[CLS] 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정 [SEP]',
 '[CLS] 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다 [SEP]',
 '[CLS] 막 걸음마 뗀 세부터 초등학교 학년생인 살용영화.ㅋㅋㅋ...별반개도 아까움. [SEP]',
 '[CLS] 원작의 긴장감을 제대로 살려내지못했다. [SEP]',
 '[CLS] 별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네 [SEP]',
 '[CLS] 액션이 없는데도 재미 있는 몇안되는 영화 [SEP]',
 '[CLS] 왜케 평점이 낮은건데 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나 [SEP]']

In [ ]:
# 라벨 추출
labels = train['label'].values
labels

array([0, 1, 0, ..., 0, 1, 0])

In [ ]:
# koELECTRA의 토크나이저로 문장을 토큰으로 분리
tokenizer = AutoTokenizer.from_pretrained('monologg/koelectra-base-v3-discriminator')
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print(sentences[0])
print(tokenized_texts[0])


[CLS] 아 더빙.. 진짜 짜증나네요 목소리 [SEP]
['[CLS]', '아', '더빙', '.', '.', '진짜', '짜증', '##나', '##네', '##요', '목소리', '[SEP]']


In [ ]:
# 문장 최대 길이 도출
lengths = []
for token in tokenized_texts:
    lengths.append(len(token))
print(max(lengths))

142


In [ ]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 160

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
print(input_ids[0])

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
print(input_ids[0])

[2, 3079, 33345, 18, 18, 7082, 13215, 4065, 4116, 4150, 6933, 3]
[    2  3079 33345    18    18  7082 13215  4065  4116  4150  6933     3
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     

In [ ]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2018, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])
print(train_inputs.shape)  # train 데이터 개수
print(validation_inputs.shape)  # validation 데이터 개수

tensor([    2,  2128,  4221,  4007,  4639,  5427,  3249,  6913,  9182,  4086,
         3249,  4820,  4244,  4189,  4353,  4366,  4149,  4441,  4814,  4216,
         2780,  4181, 12374,  4271,  4151,  4338,  4181,  4645,  3311,  4112,
         7853,  5164,  4031,  4086, 14227,  4161,  4106,  6726,  4073,  4129,
        28942,  4279,  4034,  4216,  2780,  4181, 30604,  4956,  2024,  4031,
         4086, 14227,    18,    18,    18,  6844, 12916,  8002, 21357,  4151,
        12919,  4153,  7593, 24008,  4176,  4172,  8575,  4441,  4814,  4216,
         2780,  4181,  4254,  4785, 10728,  9584,  6913,  6243,  4086,  3123,
         4200,  4034,  3083,  4176,     3,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [ ]:
# 배치 사이즈
batch_size = 64

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

<br>
<br>

# **전처리 - 테스트셋**

In [ ]:
# 리뷰 문장 추출
sentences = test['document']
sentences[:10]

0                                                   굳 ㅋ
2                  뭐야 이 평점들은.... 나쁘진 않지만 점 짜리는 더더욱 아니잖아
3                      지루하지는 않은데 완전 막장임... 돈주고 보기에는....
4           만 아니었어도 별 다섯 개 줬을텐데.. 왜 로 나와서 제 심기를 불편하게 하죠
5                                     음악이 주가 된 최고의 음악영화
6                                               진정한 쓰레기
7               마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가고개를 젖게한다
8     갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한남...
9             이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 모든 사람이 그렇지는 않네..
10                                     괜찮네요오랜만포켓몬스터잼밌어요
Name: document, dtype: object

In [ ]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 굳 ㅋ [SEP]',
 '[CLS] 뭐야 이 평점들은.... 나쁘진 않지만 점 짜리는 더더욱 아니잖아 [SEP]',
 '[CLS] 지루하지는 않은데 완전 막장임... 돈주고 보기에는.... [SEP]',
 '[CLS] 만 아니었어도 별 다섯 개 줬을텐데.. 왜 로 나와서 제 심기를 불편하게 하죠 [SEP]',
 '[CLS] 음악이 주가 된 최고의 음악영화 [SEP]',
 '[CLS] 진정한 쓰레기 [SEP]',
 '[CLS] 마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가고개를 젖게한다 [SEP]',
 '[CLS] 갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한남무 아 그립다 동사서독같은 영화가 이건 류아류작이다 [SEP]',
 '[CLS] 이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 모든 사람이 그렇지는 않네.. [SEP]',
 '[CLS] 괜찮네요오랜만포켓몬스터잼밌어요 [SEP]']

In [ ]:
# 라벨 추출
labels = test['label'].values
labels

array([1, 0, 0, ..., 0, 0, 0])

In [ ]:
# koELECTRA의 토크나이저로 문장을 토큰으로 분리
tokenizer = AutoTokenizer.from_pretrained('monologg/koelectra-base-v3-discriminator')
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print(sentences[0])
print(tokenized_texts[0])

[CLS] 굳 ㅋ [SEP]
['[CLS]', '굳', 'ㅋ', '[SEP]']


In [ ]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 160

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   2, 2104,  287,    3,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [ ]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([   2, 2104,  287,    3,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,   

In [ ]:
# 배치 사이즈
batch_size = 64

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

<br>
<br>

# **모델 생성**

In [ ]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


In [ ]:
# 분류를 위한 BERT 모델 생성
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels=2)
model.cuda()

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [ ]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 10

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

<br>
<br>

# **모델 학습**

In [ ]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 10 ========
Training...
  Batch   500  of  2,042.    Elapsed: 0:04:25.
  Batch 1,000  of  2,042.    Elapsed: 0:08:50.
  Batch 1,500  of  2,042.    Elapsed: 0:13:16.
  Batch 2,000  of  2,042.    Elapsed: 0:17:41.

  Average training loss: 0.29
  Training epcoh took: 0:18:04

Running Validation...
  Accuracy: 0.90
  Validation took: 0:00:38

======== Epoch 2 / 10 ========
Training...
  Batch   500  of  2,042.    Elapsed: 0:04:26.
  Batch 1,000  of  2,042.    Elapsed: 0:08:52.
  Batch 1,500  of  2,042.    Elapsed: 0:13:17.
  Batch 2,000  of  2,042.    Elapsed: 0:17:43.

  Average training loss: 0.21
  Training epcoh took: 0:18:05

Running Validation...
  Accuracy: 0.90
  Validation took: 0:00:38

======== Epoch 3 / 10 ========
Training...
  Batch   500  of  2,042.    Elapsed: 0:04:26.
  Batch 1,000  of  2,042.    Elapsed: 0:08:51.
  Batch 1,500  of  2,042.    Elapsed: 0:13:16.
  Batch 2,000  of  2,042.    Elapsed: 0:17:41.

  Average training loss: 0.16
  Training epco

<br>
<br>

# **테스트셋 평가**

In [ ]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch   100  of    763.    Elapsed: 0:00:17.
  Batch   200  of    763.    Elapsed: 0:00:33.
  Batch   300  of    763.    Elapsed: 0:00:50.
  Batch   400  of    763.    Elapsed: 0:01:06.
  Batch   500  of    763.    Elapsed: 0:01:23.
  Batch   600  of    763.    Elapsed: 0:01:39.
  Batch   700  of    763.    Elapsed: 0:01:56.

Accuracy: 0.90
Test took: 0:02:06


<br>
<br>

# **새로운 문장 테스트**

In [ ]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 160

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [ ]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [ ]:
logits = test_sentences(['주연배우가 아깝다. 총체적 난국...'])

print(logits)
print(np.argmax(logits))

[[ 4.0907025 -4.1015425]]
0


In [ ]:
logits = test_sentences(['최고의 영화'])

print(logits)
print(np.argmax(logits))

[[ 1.7728959 -1.8146783]]
0


<br>
<br>

# **모델 저장 및 로드**

In [ ]:
torch.save(model, './gdrive/MyDrive/sentiment_analysis/koELECTRA_nsmc_10epochs.pt')  # 저장
# model = torch.load('./gdrive/MyDrive/sentiment_analysis/bert_nsmc.pt')  # 로드

In [ ]:
os.listdir('./gdrive/MyDrive/sentiment_analysis')

['friends_train.json',
 'friends_dev.json',
 'friends_test.json',
 'ko_data.csv',
 'koELECTRA_nsmc_v2.pt',
 'koELECTRA_nsmc_v3.pt',
 'ELECTRA_friends_multi_classification.ipynb',
 'koELECTRA_nsmc_v4.pt',
 'sample.csv',
 'en_data.csv',
 'BERT_friends.pt',
 'friends_sample.csv',
 'koELECTRA_nsmc_v5.pt',
 'nsmc_sample.csv',
 'BERT_friends_sentiment_analysis.ipynb',
 'KoELECTRA_nsmc_sentiment_analysis.ipynb',
 'koELECTRA_nsmc_10epochs.pt']

## **모델 로드 후 predict**

In [ ]:
model = torch.load('./gdrive/MyDrive/sentiment_analysis/koELECTRA_nsmc_10epochs.pt')

In [ ]:
# kaggle 경진대회용 파일 변환
df = pd.read_csv('gdrive/MyDrive/sentiment_analysis/ko_data.csv', encoding='CP949')
df.head()

,Id,Sentence
0,0,정말 많이 울었던 영화입니다.
1,1,시간 낭비예요.
2,2,포스터를 저렇게밖에 만들지 못했던 제작자의 소심함에 침을 뱉고 싶다.
3,3,지금 봐도 재미있는 영화!!! 코믹과 감동!!! 그리고 요리!!!
4,4,이걸 영화로 만드는 거야?얼마나 가는지 보자.


In [ ]:
def sentence_preprocessing(sentence):
    # spaced_sent = spacing(sentence)
    only_korean_sent = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣!.~;^ ]', '', sentence)

    if len(only_korean_sent) == 0:
      pass

    result = repeat_normalize(only_korean_sent, num_repeats=2)

    return result

In [ ]:
def sent2label(sentence):
    logits = test_sentences([sentence])
    label = np.argmax(logits)

    return label

In [ ]:
df['Predicted'] = df['Sentence'].apply(sentence_preprocessing)
df['Label'] = df['Predicted'].apply(sent2label)
df.head()

,Id,Sentence,Predicted,Label
0,0,정말 많이 울었던 영화입니다.,정말 많이 울었던 영화입니다.,1
1,1,시간 낭비예요.,시간 낭비예요.,0
2,2,포스터를 저렇게밖에 만들지 못했던 제작자의 소심함에 침을 뱉고 싶다.,포스터를 저렇게밖에 만들지 못했던 제작자의 소심함에 침을 뱉고 싶다.,0
3,3,지금 봐도 재미있는 영화!!! 코믹과 감동!!! 그리고 요리!!!,지금 봐도 재미있는 영화!!! 코믹과 감동!!! 그리고 요리!!!,1
4,4,이걸 영화로 만드는 거야?얼마나 가는지 보자.,이걸 영화로 만드는 거야얼마나 가는지 보자.,0


In [ ]:
temp_df = df[['Id', 'Label']]
temp_df.columns = ['Id', 'Predicted']
temp_df.head()

,Id,Predicted
0,0,1
1,1,0
2,2,0
3,3,1
4,4,0


In [ ]:
temp_df.to_csv('gdrive/MyDrive/sentiment_analysis/nsmc_sample.csv', index=False)

In [ ]:
test_df = pd.read_csv('gdrive/MyDrive/sentiment_analysis/nsmc_sample.csv')
test_df.head()
# test_df.shape

,Id,Predicted
0,0,1
1,1,0
2,2,0
3,3,1
4,4,0
